In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from activation_function import Tanh, Linear, ReLU, Sigmoid
from layer import Layer
from mlp import MLP
from metrics import Metrics
from losses import MeanSquaredError

## Solve XOR

In [ ]:
input = [[0, 0], [0, 1], [1, 0], [1, 1]]
oracle = [[0], [1], [1], [0]]

mlp = MLP()
metrics = Metrics()
act_func1 = ReLU()
act_func2 = Sigmoid()
lr = 0.1

mlp.add(Layer(5, act_func=act_func1, n_inputs=len(input[0])))
mlp.add(Layer(len(oracle[0]), act_func=act_func2, n_inputs=5))

mlp.compile(lr, MeanSquaredError())

mlp.fit(input, oracle, epochs=2000)

for i in range(len(input)):
    out = mlp.run(input[i])
    print(f'case {i}: {np.round(out), oracle[i]}')

## SOLVE IRIS

## Dataset

In [ ]:
import pandas as pd

data = pd.read_csv('archive/Iris.csv')
data.head(5)

In [ ]:
data.describe()

In [ ]:
data['Species'].unique()

In [ ]:
data.loc[data["Species"]=="Iris-setosa","Species"]      =   0.1
data.loc[data["Species"]=="Iris-versicolor","Species"]  =   0.5
data.loc[data["Species"]=="Iris-virginica","Species"]   =   0.9

#data.drop(data[data['Species'] == 2].index, inplace=True)

data=data.iloc[np.random.permutation(len(data))]
data.head()


In [ ]:
from utils import get_splits
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_scaled = scaler.fit_transform(data.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=data.columns.values)
del df_scaled['Species']
del df_scaled['Id']
df_scaled = df_scaled.assign(Species=data['Species'].values)

df_train, df_test, _ = get_splits(df_scaled, 0.8, 0.2)
print(df_train ,len(df_train))
print(df_test, len(df_test))

In [ ]:
def parseOracle(values):
    out=[]
    for i in range(len(values)):
        if values[i] == 0:
            out.append(np.array([1,0,0]))
        elif values[i] == 1:
            out.append(np.array([1,1,0]))
        else:
            out.append(np.array([1,1,1]))
    return out

In [ ]:
X_train = df_train.iloc[ : , :4].values
y_train = df_train.iloc[:,4:].values
print(X_train[:3], y_train[3:])

## Model

In [ ]:
layers = [
    Layer(
        n_perceptrons=len(X_train[0]),
        n_inputs=len(X_train[0]),
        act_func=ReLU(),
        #kernel_regularizer=0.02,
        #bias_regularizer=0.01,
        momentum=0.5
    ),
    Layer(
        n_perceptrons=len(y_train[0]),
        n_inputs=len(X_train[0]),
        act_func=Sigmoid(),
        #kernel_regularizer=0.02,
        #bias_regularizer=0.01,
        momentum=0.5
    )
]

model = MLP(layers)

epochs, lr = 1000, 0.1
loss = MeanSquaredError()

model.compile(lr,loss)
model.fit(X_train, y_train, epochs=epochs)

In [ ]:
X_test = df_test.iloc[ : , :4].values
y_test = df_test.iloc[:,4:].values

def parse(out):
    if out > 0.75:
        return 0.9
    elif out < 0.25:
        return 0.1
    else:
        return 0.5
error = 0  
for i in range(len(X_test)):
    out = model.run(X_test[i])
    print(f'case {i}: {parse(out)}, {y_test[i][0]}')
    if parse(out) != y_test[i][0]:
        error += 1
print("error: ", error)